Our preliminary tests showed, that Optigrid performed well on Data of November 2018. In this notebook we will explore a few other months and look for similarities and descrepancies in the results. We will use preprocessed data, where things like source stability and voltage breakdowns are indicated. Moreover, for now we will limit ourselfs to stable running sources, i.e. time periods with a low variance and a high current in the BCT25. We use the already preprocessed datasets.

### Module loading
We use the Python modules from the ionsrcopt package that will be loaded in the next cells.

In [1]:
%run ../ionsrcopt/import_notebooks/Setup.ipynb

In [2]:
%run ../ionsrcopt/import_notebooks/Clustering.ipynb

First, we need to specifiy all the columns we are interested in. There are three types: Parameters, these are the ones that will be clustered later on, Measurments and columns from preprocessing.

In [4]:
time = [SourceFeatures.TIMESTAMP]
parameters = [
        SourceFeatures.BIASDISCAQNV, 
        SourceFeatures.GASAQN, 
        SourceFeatures.OVEN1AQNP,
        SourceFeatures.SAIREM2_FORWARDPOWER,
        SourceFeatures.SOLINJ_CURRENT,
        SourceFeatures.SOLCEN_CURRENT,
        SourceFeatures.SOLEXT_CURRENT,
        SourceFeatures.SOURCEHTAQNI,
        SourceFeatures.BCT25_CURRENT]
measurements = []
preprocessing = [
        ProcessingFeatures.SOURCE_STABILITY, 
        ProcessingFeatures.HT_VOLTAGE_BREAKDOWN, 
        ProcessingFeatures.DATAPOINT_DURATION,
        ProcessingFeatures.SOURCE_RUNNING]

columns_to_load = time + parameters + measurements + preprocessing

Next, specify the important files..

In [5]:
input_folder = '../Data_Preprocessed/'
input_files = ['Jan2018.csv', 'Feb2018.csv', 'Mar2018.csv', 'Apr2018.csv', 'May2018.csv', 'Jun2018.csv', 'Jul2018.csv', 'Aug2018.csv', 'Sep2018.csv', 'Oct2018.csv', 'Nov2018.csv']
input_paths = [input_folder + f for f in input_files]
output_folder = '../Data_Clustered/'
output_file = 'JanNov2018.csv'
output_path = output_folder + output_file

cluster_logfile = output_folder + 'cluster_runs.log'

...and load them.

In [ ]:
df_total = read_data_from_csv(input_paths, columns_to_load, None)
df_total = fill_columns(df_total, None, fill_nan_with_zeros=True)
df_total = convert_column_types(df_total)
df_total.memory_usage()

In [7]:
df_total.shape

(5824993, 13)

Now we select what data we are interested in.

In [20]:
def select_values(df_total, parameters, selector):
    data = df_total.loc[selector, parameters].values
    weights = df_total.loc[selector, ProcessingFeatures.DATAPOINT_DURATION].values
    return data, weights

Once the data is ready we can begin clustering. But first we standard scale it, so that all parameters have the same variance.

In [21]:
from sklearn import preprocessing

def scale_values(values, scaler):
    if not scaler:
        scaler = preprocessing.StandardScaler().fit(values)
    values_scaled = scaler.transform(values)
    return scaler, values_scaled

The parameters for optigrid can be chosen by visually examening the distribution of normalized data, see below.

In [22]:
d=len(parameters)
q=1
max_cut_score = 0.1
noise_level = 0.05

optigrid_params = {
    'd' : d, 
    'q' : q, 
    'max_cut_score' : max_cut_score, 
    'noise_level' : noise_level,
    'kde_bandwidth' : 0.06,
    'verbose' : True}

In [23]:
def run_optigrid(values_scaled, weights, optigrid_params):
    optigrid = Optigrid(**optigrid_params)
    optigrid.fit(values_scaled, weights)
    return optigrid

Once the clusters are found, we set an according column in the original dataframe containing all data.

In [24]:
def assign_clusters_df_total(df_total, optigrid, num_values, selector):
    clusters = np.zeros(num_values)

    for i, cluster in enumerate(optigrid.clusters):
        clusters[cluster] = i
    df_total.loc[selector, ProcessingFeatures.CLUSTER] = clusters

And here we bundle all these steps together.

In [31]:
def cluster(df_total, parameters, source_stable, optigrid_params):
    print("Starting clustering for source stability {}".format(source_stable))
    source_stability = df_total[ProcessingFeatures.SOURCE_STABILITY] == source_stable
    voltage_breakdown_selection = df_total[ProcessingFeatures.HT_VOLTAGE_BREAKDOWN] > 0
    source_running = df_total[ProcessingFeatures.SOURCE_RUNNING] == True
    
    selector = source_stability & ~voltage_breakdown_selection & source_running
    values, weights = select_values(df_total, parameters, selector) # First, get the data without breakdowns,
    scaler, values_scaled = scale_values(values, None) # standard scale it
    optigrid = run_optigrid(values_scaled, weights, optigrid_params) # and compute the clusters.
    assign_clusters_df_total(df_total, optigrid, len(values), selector) # Then, assign the found clusters to the original dataframe in a new column 'optigrid_clusters'
    
    print("Scoring voltage breakdowns")
    selector = source_stability & voltage_breakdown_selection & source_running
    values, weights = select_values(df_total, parameters, selector) # Now, get the datapoints when the voltage broke down
    _, values_scaled = scale_values(values, scaler) # scale it to the same ranges
    scored_samples = optigrid.score_samples(values_scaled) # and find the corresponding clusters.
    df_total.loc[selector, ProcessingFeatures.CLUSTER] = scored_samples

In [32]:
df_total[ProcessingFeatures.CLUSTER] = -1
cluster(df_total, parameters, 1, optigrid_params)
cluster(df_total, parameters, 0, optigrid_params)

Starting clustering for source stability 1
Found following cuts: [(-1.5586662075736306, 2, 3.5424831913455504e-18)]
Evaluating subgrid: 100.00% of datapoints
Found following cuts: [(-0.536211042693167, 1, 8.140809629402368e-38)]
Evaluating subgrid: 4.99% of datapoints
Found cluster 0
Evaluating subgrid: 4.99% of datapoints
Found following cuts: [(0.4716584670423257, 1, 0.0026910531611852224)]
Evaluating subgrid: 3.04% of datapoints
Found cluster 1
Evaluating subgrid: 3.04% of datapoints
Found cluster 2
Found cluster: 3.04% of datapoints
Found cluster: 4.99% of datapoints
Evaluating subgrid: 100.00% of datapoints
Found following cuts: [(2.576037734445899, 5, 5.1615825937021344e-18)]
Evaluating subgrid: 95.01% of datapoints
Found following cuts: [(-2.4684997495978767, 3, 5.357567430109741e-43)]
Evaluating subgrid: 90.66% of datapoints
Found following cuts: [(0.6367034912109375, 5, 3.542028702909208e-05)]
Evaluating subgrid: 6.87% of datapoints
Found following cuts: [(-1.4965935186906294,

Found following cuts: [(-0.25164432959123095, 5, 0.0036273242021894175)]
Evaluating subgrid: 10.79% of datapoints
Found following cuts: [(-0.36382837548400415, 2, 0.0006747717591545009)]
Evaluating subgrid: 7.22% of datapoints
Found cluster 38
Evaluating subgrid: 7.22% of datapoints
Found following cuts: [(-1.2289728219009408, 4, 0.0047451462272256155)]
Evaluating subgrid: 4.84% of datapoints
Found cluster 39
Evaluating subgrid: 4.84% of datapoints
Found following cuts: [(1.0529433639362604, 6, 0.004247254585464666)]
Evaluating subgrid: 3.83% of datapoints
Found following cuts: [(-0.8960783507185752, 4, 0.004338945812992136)]
Evaluating subgrid: 2.23% of datapoints
Found cluster 40
Evaluating subgrid: 2.23% of datapoints
Found cluster 41
Found cluster: 2.23% of datapoints
Evaluating subgrid: 3.83% of datapoints
Found cluster 42
Found cluster: 3.83% of datapoints
Found cluster: 4.84% of datapoints
Found cluster: 7.22% of datapoints
Evaluating subgrid: 10.79% of datapoints
Found followin

Found cluster 18
Evaluating subgrid: 2.15% of datapoints
Found cluster 19
Found cluster: 2.15% of datapoints
Evaluating subgrid: 4.22% of datapoints
Found cluster 20
Found cluster: 4.22% of datapoints
Found cluster: 5.93% of datapoints
Evaluating subgrid: 8.88% of datapoints
Found following cuts: [(-0.31087365355154484, 1, 0.04921032474141101)]
Evaluating subgrid: 2.94% of datapoints
Found cluster 21
Evaluating subgrid: 2.94% of datapoints
Found following cuts: [(-0.1349804595564351, 1, 0.06378909100855813)]
Evaluating subgrid: 1.83% of datapoints
Found cluster 22
Evaluating subgrid: 1.83% of datapoints
Found cluster 23
Found cluster: 1.83% of datapoints
Found cluster: 2.94% of datapoints
Found cluster: 8.88% of datapoints
Evaluating subgrid: 10.62% of datapoints
Found cluster 24
Found cluster: 10.62% of datapoints
Found cluster: 18.25% of datapoints
Found cluster: 31.05% of datapoints
Found cluster: 32.52% of datapoints
Evaluating subgrid: 91.19% of datapoints
Found following cuts: [(

Found cluster 61
Found cluster: 3.24% of datapoints
Found cluster: 6.06% of datapoints
Evaluating subgrid: 7.61% of datapoints
Found cluster 62
Found cluster: 7.61% of datapoints
Found cluster: 30.75% of datapoints
Found cluster: 33.44% of datapoints
Found cluster: 44.28% of datapoints
Found cluster: 48.05% of datapoints
Evaluating subgrid: 50.13% of datapoints
Found following cuts: [(0.595610175469909, 3, 0.06295910641462461)]
Evaluating subgrid: 2.08% of datapoints
Found cluster 63
Evaluating subgrid: 2.08% of datapoints
Found cluster 64
Found cluster: 2.08% of datapoints
Found cluster: 50.13% of datapoints
Found cluster: 51.84% of datapoints
Found cluster: 55.60% of datapoints
Found cluster: 58.67% of datapoints
Found cluster: 91.19% of datapoints
Found cluster: 93.13% of datapoints
Evaluating subgrid: 95.14% of datapoints
Found following cuts: [(3.3349704453439424, 0, 7.389693697606996e-05)]
Evaluating subgrid: 2.01% of datapoints
Found cluster 65
Evaluating subgrid: 2.01% of datap

#### Long term storage
We will save the clustered data to a file.

First, create the logging string.

In [33]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d.%m.%Y %H:%M:%S")

logstring = "[{}] \'{}\' cluster results saved to \'{}\'. Columns used: {}. Parameters used: {}\n".format(dt_string, input_paths, output_path, parameters, optigrid_params)
with open(cluster_logfile, "a") as myfile:
    myfile.write(logstring)

logstring

"[11.12.2019 12:33:49] '['../Data_Preprocessed/Jan2018.csv', '../Data_Preprocessed/Feb2018.csv', '../Data_Preprocessed/Mar2018.csv', '../Data_Preprocessed/Apr2018.csv', '../Data_Preprocessed/May2018.csv', '../Data_Preprocessed/Jun2018.csv', '../Data_Preprocessed/Jul2018.csv', '../Data_Preprocessed/Aug2018.csv', '../Data_Preprocessed/Sep2018.csv', '../Data_Preprocessed/Oct2018.csv', '../Data_Preprocessed/Nov2018.csv']' cluster results saved to '../Data_Clustered/JanNov2018.csv'. Columns used: ['IP.NSRCGEN:BIASDISCAQNV', 'IP.NSRCGEN:GASAQN', 'IP.NSRCGEN:OVEN1AQNP', 'IP.SAIREM2:FORWARDPOWER', 'IP.SOLINJ.ACQUISITION:CURRENT', 'IP.SOLCEN.ACQUISITION:CURRENT', 'IP.SOLEXT.ACQUISITION:CURRENT', 'IP.NSRCGEN:SOURCEHTAQNI', 'ITF.BCT25:CURRENT']. Parameters used: {'d': 9, 'q': 1, 'max_cut_score': 0.1, 'noise_level': 0.05, 'kde_bandwidth': 0.06, 'verbose': True}\n"

Now we can save the dataframe to a file.

In [34]:
df_total = df_total.astype({ProcessingFeatures.CLUSTER : 'int64'})
df_total[df_total.shift(1)==df_total] = np.nan
df_total.to_csv(output_path)

In [ ]:
cluster_performance_dbi(values_scaled, optigrid.clusters, optigrid.num_clusters)

In [ ]:
def silhouette_coefficient(a, b):
    if a < b:
        return 1 - a/b
    elif a == b:
        return 0
    else:
        return b/a - 1
    
def cluster_performance_silhouette(df_total, values_scaled, clusters, source_stability, voltage_breakdown_selection, num_clusters):
    mean_distances = np.array([np.array([np.sum(np.linalg.norm(values_scaled[cluster]-x, axis=1)) / len(cluster) for cluster in clusters]) for x in values_scaled])
    optigrid_cluster = df_total.loc[source_stability & voltage_breakdown_selection, 'optigrid_cluster']
    selector = np.ones((len(values_scaled), num_clusters), dtype=bool)
    selector[range(len(values)), optigrid_cluster] = False
    print(mean_distances)
    print(optigrid_cluster)
    print(selector)
    print(np.ma.masked_array(mean_distances, ~selector))
    df_total.loc[source_stability & voltage_breakdown_selection, 'mean_dist_same_cluster'] = np.amin(np.ma.masked_array(mean_distances, selector), axis=1)
    df_total.loc[source_stability & voltage_breakdown_selection, 'min_mean_dist_different_cluster'] = np.amin(np.ma.masked_array(mean_distances, ~selector), axis=1)
    df_total.loc[source_stability & voltage_breakdown_selection, 'silhouette'] = np.vectorize(silhouette_coefficient)(df_total.loc[source_stability & voltage_breakdown_selection, 'mean_dist_same_cluster'], df_total.loc[source_stability & voltage_breakdown_selection, 'min_mean_dist_different_cluster'])

In [ ]:
def all_pairs_euclid_squared_numpy(A, B):
    sqrA = np.broadcast_to(np.sum(np.power(A, 2), 1).reshape(A.shape[0], 1), (A.shape[0], B.shape[0]))
    sqrB = np.broadcast_to(np.sum(np.power(B, 2), 1).reshape(B.shape[0], 1), (B.shape[0], A.shape[0])).transpose()

    return sqrA - 2*np.matmul(A, B.transpose()) + sqrB

def cluster_performance_dbi(values_scaled, clusters, num_clusters):
    print("values_scaled: {}".format(values_scaled))
    values_per_cluster = [np.take(values_scaled, c, axis=0) for c in clusters]
    means = np.array([np.mean(c, axis=0) for c in values_per_cluster])
    print("values_per_cluster: {}".format(values_per_cluster[0][:10]))
    print("means: {}".format(means))
    assigned_cluster_mean = np.zeros((len(values_scaled), len(values_scaled[0])))
    for i, c in enumerate(clusters):
        assigned_cluster_mean[c] = means[i]
    print("assigned_cluster_mean: {}".format(assigned_cluster_mean))
        
    dists_from_means = np.linalg.norm(values_scaled-assigned_cluster_mean, axis=1)
    print("dists_from_means: {}".format([dists_from_means[c] for c in clusters]))
    s = np.array([np.sqrt(1./len(c) * np.sum(dists_from_means[c])) for c in clusters])
    print("s: {}".format(s))
    
    dists_between_clusters = all_pairs_euclid_squared_numpy(means, means)
    np.fill_diagonal(dists_between_clusters, np.nan)
    print("dists_between_clusters: {}".format(dists_between_clusters))
    
    r = np.tile(s, (num_clusters, 1))
    r = (r + r.T) / dists_between_clusters
    print("r: {}".format(r))
    d = np.nanmax(r, axis=1)
    dbi = np.mean(d)
    print("Davies-Bouldin index per cluster: {}".format(d))
    print("Davies-Bouldin index total: {}".format(dbi))

In [ ]:
def describe_clusters(optigrid, data, parameters):
    values = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
    result = pd.DataFrame(columns = pd.MultiIndex.from_tuples([(p, v) for p in parameters for v in values] + [('DENSITY', 'count'), ('DENSITY', 'percentage')]))
    result.index.name = 'OPTIGRID_CLUSTER'
    
    for i, cluster in enumerate(optigrid.clusters):
        cluster_data = np.take(data, cluster, axis=0)
        mean = np.mean(cluster_data, axis=0)
        std = np.std(cluster_data, axis=0)
        quantiles = np.quantile(cluster_data, [0, 0.25, 0.5, 0.75, 1], axis=0)
        cluster_description = [[mean[i], std[i], quantiles[0][i], quantiles[1][i], quantiles[2][i], quantiles[3][i], quantiles[4][i]] for i in range(len(parameters))]
        cluster_description = [item for sublist in cluster_description for item in sublist]
        cluster_description.append(len(cluster))
        cluster_description.append(len(cluster)/len(data)*100)
        result.loc[i] = cluster_description
    return result

described = describe_clusters(optigrid, data, parameters)

In [ ]:
pd.set_option('display.max_columns', 500)
wanted_statistics = [[(param, 'mean'), (param, 'std')] for param in parameters]
wanted_statistics = [item for sublist in wanted_statistics for item in sublist] + [('DENSITY', 'percentage')]

num_of_clusters_to_print = 10
described.sort_values(by=[('DENSITY', 'percentage')], ascending=False, inplace = True)
print("Sum of densities of printed clusters: {:.1f}%".format(described.head(n=num_of_clusters_to_print)[('DENSITY', 'percentage')].sum()))
described.head(n=num_of_clusters_to_print)[wanted_statistics].round(3)

For visualizing the clusters we will plot the densities of the parameters. For comparability we will use explicit ranges for the x-axis per parameter. Those ranges should be chosen beforehand by an expert to validate or falsify his intuition.

In [ ]:
num_clusters = 6 # number of clusters to visualize
data = df[parameters].values # We select the unscaled data again, because by clustering we did not change any ordering and this data corresponds to the real world
num_datapoints = len(data)

resolution = 200
bandwidth = [1, 0.01, 1, 10, 0.1, 0.001]
num_kde_samples = 40000

parameter_ranges = [[0,0] for i in range(len(parameters))]
parameter_ranges[0] = [-300, -200] # Biasdisc x-axis

parameter_ranges[1] = [5.1, 5.3] # Gas x-axis
#parameter_ranges[2] = [0, 3] # High voltage current x-axis
parameter_ranges[2] = [200, 300] # SolCen current x-axis
#parameter_ranges[3] = [900, 2100] # Forwardpower x-axis
parameter_ranges[3] = [1200, 1300] # SolExt current x-axis
parameter_ranges[4] = [5, 20] # Oven1 power x-axis
parameter_ranges[5] = [0, 0.05] # BCT25 current x-axis

best_clusters = sorted(optigrid.clusters, key=lambda x: len(x), reverse=True)
for i, cluster in enumerate(best_clusters[:num_clusters]):
    median = [described.iloc[i,described.columns.get_loc((param, '50%'))] for param in parameters]
    plot_cluster(data, cluster, parameters, parameter_ranges, resolution=resolution, median=median, bandwidth=bandwidth, percentage_of_values=1, num_kde_samples=num_kde_samples)

Now, we want to find all high voltage breakdowns that correspond to the currently considered source stability, and find out to which cluster each datapoint belongs.

In [ ]:
wanted_statistics.append(('num_of_breakdowns', ''))
described.head(n=num_of_clusters_to_print)[wanted_statistics].round(3)

In [ ]:
wanted_statistics = [[(param, 'mean')] for param in parameters]
wanted_statistics = [item for sublist in wanted_statistics for item in sublist] + [('num_of_breakdowns', '')]
corr_described = described[wanted_statistics].corr()
corr_described.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

In [ ]:
pd.set_option('display.max_columns', 500)
wanted_statistics = [[(param, 'mean'), (param, 'std'),  (param, 'min'),  (param, 'max')] for param in parameters]
wanted_statistics = [item for sublist in wanted_statistics for item in sublist]
df_breakdowns.groupby('is_breakdown').describe()[wanted_statistics].round(2)

In [ ]:
import numpy as np

def d(x,y):
    return np.linalg.norm(x-y)

size = 10
data = np.random.uniform(0, 1, (size, 1))
data

In [ ]:
a = np.array([np.sum(np.linalg.norm(data-x, axis=1)) for x in data]) / (size - 1)
a

In [ ]:
import pandas as pd
import numpy as np

values = [0, 2, 2, 2, 3, 3, 1]
values = np.array([[x, x] for x in values])
clusters = [[0, 6], [1, 2, 3], [4, 5]]

values, clusters

In [ ]:
cluster_performance_dbi(values, clusters, len(clusters))

In [ ]:
from sklearn.metrics import davies_bouldin_score

davies_bouldin_score(values, [0, 1, 1, 1, 2, 2, 0])

In [ ]:
source_stable = 1
print("Starting clustering for source stability {}".format(source_stable))
source_stability = df_total['source_stable'] == source_stable
voltage_breakdown_selection = df_total['is_breakdown'] > 0

values = select_values(df_total, parameters, source_stability, ~voltage_breakdown_selection) # First, get the data without breakdowns,
scaler, values_scaled = scale_values(values, None) # standard scale it
print(values_scaled)
optigrid = run_optigrid(values_scaled, optigrid_params) # and compute the clusters.
print(values_scaled)
#assign_clusters_df_total(df_total, optigrid, len(values), source_stability, ~voltage_breakdown_selection) # Then, assign the found clusters to the original dataframe in a new column 'optigrid_clusters'
print("Calculating cluster performance cluster performance")
#cluster_performance_silhouette(df_total, values_scaled, optigrid.clusters, source_stability, voltage_breakdown_selection, optigrid.num_clusters)